In [1]:
import lda
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rchaks/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/rchaks/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/rchaks/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/rchaks/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [32]:
scraped_data = pd.read_csv('./republic/republic_articles_final.csv')
scraped_data['time'] = pd.to_datetime(scraped_data['time'])
scraped_data = scraped_data[["title","content"]].dropna().reset_index(drop=True).copy()
print("Number of rows with any of the empty columns:\n",scraped_data.isnull().sum())
scraped_data.fillna(" ",inplace= True)
print("Dimensions:",scraped_data.shape)
scraped_data.head()

Number of rows with any of the empty columns:
 title      0
content    0
dtype: int64
Dimensions: (2196, 2)


,title,content
0,"Why Did Kerala Have A Backlog Of 7,000 COVID-1...",In a recent update to the COVID-19 situation i...
1,Kerala Artist Creates 24-foot-long Theyyam's F...,A Kerala-based artist created an installation ...
2,'Govt as an enabler..': PM Modi Launches India...,Prime Minister Narendra Modi on Monday launche...
3,"NEET Phase 2 Registration 2021 To End Today, H...",NEET Phase 2 registration: National Testing Ag...
4,"'Leave immediately' | US, UK Issue Advisory; W...","The United States and Britain on Monday, 11 Oc..."


In [33]:
word_tokenizer=RegexpTokenizer(r'\w+')
wordnet_lemmatizer = WordNetLemmatizer()
stopwords_nltk=set(stopwords.words('english'))

In [34]:
def tokenize_text(version_desc):
    words = word_tokenizer.tokenize(wordnet_lemmatizer.lemmatize(version_desc.lower()))
    tokens = {word for word, tag in nltk.pos_tag(words) if tag in 'NN'}
    tokens = {x for x in tokens if not str(x).startswith("#")} # removing hashtags
    tokens = {x for x in tokens if str(x).isalpha()} # removing non-alpha's 
    tokens = tokens.difference(stopwords_nltk) # removing stop-words
    return tokens

In [35]:
# document = scraped_data["content"][1]
# print(document)
# tokenize_text()

In [37]:
vec_words = CountVectorizer(tokenizer=tokenize_text,decode_error='ignore')
total_features_words = vec_words.fit_transform(scraped_data["content"])
total_features_words.shape

(2196, 14171)

In [38]:
total_features_words

<2196x14171 sparse matrix of type '<class 'numpy.int64'>'
	with 129862 stored elements in Compressed Sparse Row format>

In [44]:
#checking for nulls if present any
ntopics= 100


In [45]:
model = lda.LDA(n_topics=int(ntopics), n_iter=5000, random_state=1, alpha= 10);
model.fit(total_features_words);


INFO:lda:n_documents: 2196
INFO:lda:vocab_size: 14171
INFO:lda:n_words: 129862
INFO:lda:n_topics: 100
INFO:lda:n_iter: 5000
INFO:lda:<0> log likelihood: -1857415
INFO:lda:<10> log likelihood: -1376544
INFO:lda:<20> log likelihood: -1344492
INFO:lda:<30> log likelihood: -1331555
INFO:lda:<40> log likelihood: -1325056
INFO:lda:<50> log likelihood: -1321633
INFO:lda:<60> log likelihood: -1318103
INFO:lda:<70> log likelihood: -1316564
INFO:lda:<80> log likelihood: -1314627
INFO:lda:<90> log likelihood: -1311582
INFO:lda:<100> log likelihood: -1310030
INFO:lda:<110> log likelihood: -1309814
INFO:lda:<120> log likelihood: -1306758
INFO:lda:<130> log likelihood: -1306834
INFO:lda:<140> log likelihood: -1304435
INFO:lda:<150> log likelihood: -1305934
INFO:lda:<160> log likelihood: -1305110
INFO:lda:<170> log likelihood: -1303092
INFO:lda:<180> log likelihood: -1302106
INFO:lda:<190> log likelihood: -1305690
INFO:lda:<200> log likelihood: -1301482
INFO:lda:<210> log likelihood: -1301974
INFO:ld

In [59]:
topic_word = model.topic_word_
doc_topic=model.doc_topic_
doc_topic=pd.DataFrame(doc_topic)
doc_topic["topics"] = doc_topic[[x for x in range(int(ntopics))]].apply(lambda x: (x.idxmax()), axis = 1)
doc_topic

,0,1,2,3,4,5,6,topics
0,0.107143,0.250000,0.053571,0.142857,0.017857,0.071429,0.357143,6
1,0.046729,0.163551,0.156542,0.287383,0.095794,0.032710,0.217290,3
2,0.059701,0.134328,0.298507,0.283582,0.059701,0.074627,0.089552,2
3,0.010989,0.109890,0.010989,0.461538,0.076923,0.197802,0.131868,3
4,0.050000,0.083333,0.216667,0.550000,0.050000,0.016667,0.033333,3
...,...,...,...,...,...,...,...,...
2148,0.365854,0.134146,0.097561,0.109756,0.146341,0.085366,0.060976,0
2149,0.032787,0.114754,0.032787,0.590164,0.016393,0.032787,0.180328,3
2150,0.112676,0.070423,0.112676,0.028169,0.323944,0.323944,0.028169,4
2151,0.042857,0.514286,0.028571,0.028571,0.057143,0.314286,0.014286,1


In [52]:
scraped_data=scraped_data.join(doc_topic)

scraped_data["topic"] = #scraped_data[[x for x in range(int(ntopics))]].apply(lambda x: (x.idxmax()), axis = 1)
scraped_data

,title,content,classification,0,1,2,3,4,5,6,topic
0,"Divert Arms Money For Food, Vaccines: Pope At ...",in his speech during german chancellor angela ...,pos,0.107143,0.250000,0.053571,0.142857,0.017857,0.071429,0.357143,6
1,PM Modi's 20 Years In Government: 20 Landmark ...,pm narendra modi on thursday completed 20 year...,pos,0.046729,0.163551,0.156542,0.287383,0.095794,0.032710,0.217290,3
2,Adar Poonawalla Hails UK's Decision To Scrap Q...,as the uk government announced that indian tra...,pos,0.059701,0.134328,0.298507,0.283582,0.059701,0.074627,0.089552,2
3,"'Only 2 Families Protected In Kashmir, Yet The...",srinagar mayor junaid azim mattu took an uncom...,neg,0.010989,0.109890,0.010989,0.461538,0.076923,0.197802,0.131868,3
4,Africa's Health Officials Hail New Malaria Vac...,health officials in africa are confident and h...,pos,0.050000,0.083333,0.216667,0.550000,0.050000,0.016667,0.033333,3
...,...,...,...,...,...,...,...,...,...,...,...
2148,International Emmys 2021: Nawazuddin Hails 'Se...,coming in as great news for nawazuddin siddiqu...,pos,0.365854,0.134146,0.097561,0.109756,0.146341,0.085366,0.060976,0
2149,Defence Ministry's 'Arjun' Mk-1A Tanks Will Bo...,the ministry of defence as a part of the centr...,pos,0.032787,0.114754,0.032787,0.590164,0.016393,0.032787,0.180328,3
2150,Mahant Narendra Giri's Death Case: CBI Takes O...,as per the recommendation of the uttar pradesh...,neg,0.112676,0.070423,0.112676,0.028169,0.323944,0.323944,0.028169,4
2151,Punjab CM Charanjit Channi Discusses New Cabin...,taking control of his new government punjab c...,neg,0.042857,0.514286,0.028571,0.028571,0.057143,0.314286,0.014286,1


In [53]:
for index, item in scraped_data.groupby(["topic"]):
    print("\n",index)
    for idx, stuff in item[:20].iterrows():
        print(stuff["title"])


 0
Seattle Seahawks Vs Los Angeles Rams NFL Live Streaming Details And Dream 11 Predictions
Luis Suarez Attacks Barcelona Boss Again; States Phone Call Ending His Stint Lasted 40s
India's Anshu Malik Wins Silver Medal At World Wrestling Championships; Sarita Bags Bronze
T20 World Cup: ICC Unveils Umpires For India Versus Pakistan Clash
'Aryan Khan Is Friends With Arbaaz, But Doesn't Know His Activities': SRK's Son's Lawyer
IPL 2021: Fans Impressed With KL Rahul's Aggression; Call Him 'world's Best T20 Batter'
Goswami-Fitzpatrick Trophy: Ex-spinner Wants India-Australia Series Named After Legends
SRK's Son Aryan Khan Sent To 14-days Judicial Custody In NCB Drug Raid; Bail Plea Tomorrow
Suresh Raina's Exclusion From CSK's Playing XI Against PBKS Surprises Heartbroken Fans
T20 World Cup: Match Officials Announced; Neutral Umpires Named For All Contests
Petty Pakistan Writes 'T20 World Cup UAE 2021' Instead Of 'India 2021' In Team Jersey
Kangana Ranaut Breaks Silence On Shah Rukh Khan's S

In [54]:
x = scraped_data.groupby(["topic"])["classification"].value_counts()
x = x.unstack()
x

classification,neg,pos
topic,,
0,59,270
1,137,186
2,118,149
3,254,290
4,168,60
5,182,174
6,12,94


In [55]:

plt.hist(x=x.index, weights= x["Negative"], label="Negative", rwidth= 1/3)# , align="left"
# plt.hist(x=x.index, weights= x["Neutral"], label="Neutral", rwidth= 1/3)# , align="mid"
plt.hist(x=x.index, weights= x["Positive"], label="Positive", rwidth= 1/3)# , align="right"

plt.legend()

KeyError: 'Negative'

In [ ]:
topics=pd.DataFrame(topic_word)
topics.columns=vec_words.get_feature_names()
x = topics.transpose()

x.insert(0,column="topic",value=x.apply(lambda x: (x.idxmax()), axis = 1))
x

/Users/rchaks/opt/anaconda3/envs/barua_proj/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,topic,0,1,2
aa,0,3.211666e-04,5.113774e-07,4.788806e-07
aadhar,2,5.343870e-07,5.113774e-07,4.836695e-05
aaditya,1,5.343870e-07,5.164912e-05,4.788806e-07
aadmi,1,5.343870e-07,4.607510e-04,4.788806e-07
aam,1,5.343870e-07,7.675774e-04,4.788806e-07
...,...,...,...,...
zr,1,5.343870e-07,5.164912e-05,4.788806e-07
zuckerberg,0,3.746053e-04,5.113774e-07,4.788806e-07
zulaikha,2,5.343870e-07,5.113774e-07,4.836695e-05
zydus,2,5.343870e-07,5.113774e-07,3.356953e-04


In [ ]:
for index, item in x.groupby(["topic"]):
    print(index)
    print(set(item.sort_values(by=index)[:10].index))

0
{'karat', 'kapadia', 'karve', 'kanpur', 'kaur', 'kantilal', 'kazi', 'kanoon', 'karte', 'sunset'}
1
{'listener', 'license', 'locator', 'livestock', 'lodging', 'literacy', 'aaditya', 'lito', 'lighting', 'lobby'}
2
{'mandal', 'manifesto', 'aadhar', 'manitoba', 'manhar', 'manav', 'manikyam', 'manangement', 'manipulate', 'mamta'}
